In [3]:
# Módulo 2: Preprocesamiento de Datos

In [1]:
# Importar librerías esenciales
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [2]:
# 1. Escalado y Normalización de Datos
# Escalado: Proceso de ajustar la escala de las características. Ejemplo: StandardScaler.
# Normalización: Ajustar los datos para que se encuentren en un rango específico. Ejemplo: MinMaxScaler.

In [6]:
# 2. Tratamiento de Valores Faltantes y Outliers
# Valores faltantes: Utilizar técnicas como la imputación.
# Outliers: Detectar y tratar valores atípicos que puedan afectar el modelo.

In [4]:
# 3. Encoding de Variables Categóricas
# - One-Hot Encoding: Crear variables binarias para cada categoría.
# - Label Encoding: Convertir categorías en valores numéricos.

In [5]:
# 4. Dividir Datos en Conjuntos de Entrenamiento y Prueba
# Separar los datos en conjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo.

In [7]:
# Proyecto: Preprocesamiento de datos de ventas de viviendas

# Descargar el dataset sobre precios de viviendas
url = 'https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv'
data = pd.read_csv(url)

# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
print(data.head())

Primeras filas del dataset:
      crim    zn  indus  chas    nox     rm   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

        b  lstat  medv  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
4  396.90   5.33  36.2  


In [8]:
# 4.1. Limpieza de Datos
# Verificar valores faltantes
print("\nValores faltantes en cada columna:")
print(data.isnull().sum())

# Imputar valores faltantes (si los hubiera)
imputer = SimpleImputer(strategy='median')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)


Valores faltantes en cada columna:
crim       0
zn         0
indus      0
chas       0
nox        0
rm         0
age        0
dis        0
rad        0
tax        0
ptratio    0
b          0
lstat      0
medv       0
dtype: int64


In [9]:
# Identificación y tratamiento de outliers
# Para simplificar, se usará una técnica básica para detectar outliers mediante el IQR (Rango Intercuartílico).
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[col] >= (Q1 - 1.5 * IQR)) & (df[col] <= (Q3 + 1.5 * IQR))]

In [10]:
for column in data.columns:
    data_imputed = remove_outliers(data_imputed, column)

In [11]:
# 4.2. Encoding de Variables Categóricas
# Identificar variables categóricas y numéricas
categorical_features = [col for col in data_imputed.select_dtypes(include=['object']).columns]
numerical_features = [col for col in data_imputed.select_dtypes(include=['int64', 'float64']).columns]

# Crear el pipeline para preprocesar los datos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

# Aplicar el preprocesador a los datos
X = data_imputed.drop('MEDV', axis=1)  # Suponiendo que 'MEDV' es la variable objetivo (precios de viviendas)
y = data_imputed['MEDV']

X_preprocessed = preprocessor.fit_transform(X)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

KeyError: "['MEDV'] not found in axis"

In [ ]:
print("\nDatos preprocesados y divididos para el modelado.")
print(f"Dimensiones de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Dimensiones de prueba: {X_test.shape}, {y_test.shape}")